In [10]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import pymongo
import pprint

In [2]:
# Initializing a variable with the path for chromedriver
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
# Creating a Browser instance for Google Chrome using chromedriver
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Initializing a variable with the url of the AAA website that displays fuel prices by grade by state
fuel_price_url = "https://gasprices.aaa.com/state-gas-price-averages/"
# Visiting fuel_price_url
browser.visit(fuel_price_url)

In [4]:
# Initializing a variable with the html script of fuel_price_url
html = browser.html
# Initializing a variable with the html script parsed by BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
# Initializing a variable with the list created from the html script for the fuel price table
table = pd.read_html(html)

In [5]:
# Intitalizing a DataFrame from the list "table"
state_fuel_df = table[0]
# Setting the column names of state_fuel_df
state_fuel_df.columns = ["state", "regular", "mid_grade", "premium", "diesel"]
# Displaying state_fuel_df
state_fuel_df.head()

,state,regular,mid_grade,premium,diesel
0,Alaska,$3.039,$3.127,$3.270,$2.986
1,Alabama,$2.269,$2.592,$2.915,$2.777
2,Arkansas,$2.289,$2.571,$2.840,$2.709
3,Arizona,$2.785,$3.010,$3.237,$3.009
4,California,$3.592,$3.755,$3.867,$3.906


In [6]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [7]:
# Define database and collection
db = client.etl_db
collection = db.fuel_prices
db.fuel_prices.drop()

In [8]:
# Using a for-loop to iterate through the rows of state_fuel_df and insert each state's name and its fuel prices into 
# the MongoDB database fuel_prices_db collection fuel_prices as individual documents
for index, row in state_fuel_df.iterrows():

    post = {
                "state": row["state"],
                "regular": row["regular"],
                "mid_grade": row["mid_grade"],
                "premium": row["premium"],
                "diesel": row["diesel"]
            }

    collection.insert_one(post)


In [9]:
# This is how you can list the names of collections in a MongoDB database using PyMongo
db.list_collection_names()

['fuel_prices', 'state_votes']

In [14]:
# This is how you can get a single document using PyMongo
db.fuel_prices.find_one({"state": "Alaska"})

{'_id': ObjectId('5d5c262ba989f250c582fbe5'),
 'state': 'Alaska',
 'regular': '$3.039',
 'mid_grade': '$3.127',
 'premium': '$3.270',
 'diesel': '$2.986'}

In [16]:
# This is how you can return all of the keys of a single document using PyMongo
db.fuel_prices.find_one({"state": "Alaska"}).keys()

dict_keys(['_id', 'state', 'regular', 'mid_grade', 'premium', 'diesel'])

In [31]:
# This is how you can get a single document by its ObjectId using PyMongo. You hava to import this dependency first. 
from bson.objectid import ObjectId
db.fuel_prices.find_one({"_id": ObjectId('5d5c262ba989f250c582fbe5')})

{'_id': ObjectId('5d5c262ba989f250c582fbe5'),
 'state': 'Alaska',
 'regular': '$3.039',
 'mid_grade': '$3.127',
 'premium': '$3.270',
 'diesel': '$2.986'}

In [36]:
# This is how you can print or return all of the entries in a MongoDB collection using Pymongo.
for post in db.fuel_prices.find():
...   pprint.pprint(post)

{'_id': ObjectId('5d5c262ba989f250c582fbe5'),
 'diesel': '$2.986',
 'mid_grade': '$3.127',
 'premium': '$3.270',
 'regular': '$3.039',
 'state': 'Alaska'}
{'_id': ObjectId('5d5c262ca989f250c582fbe6'),
 'diesel': '$2.777',
 'mid_grade': '$2.592',
 'premium': '$2.915',
 'regular': '$2.269',
 'state': 'Alabama'}
{'_id': ObjectId('5d5c262ca989f250c582fbe7'),
 'diesel': '$2.709',
 'mid_grade': '$2.571',
 'premium': '$2.840',
 'regular': '$2.289',
 'state': 'Arkansas'}
{'_id': ObjectId('5d5c262ca989f250c582fbe8'),
 'diesel': '$3.009',
 'mid_grade': '$3.010',
 'premium': '$3.237',
 'regular': '$2.785',
 'state': 'Arizona'}
{'_id': ObjectId('5d5c262ca989f250c582fbe9'),
 'diesel': '$3.906',
 'mid_grade': '$3.755',
 'premium': '$3.867',
 'regular': '$3.592',
 'state': 'California'}
{'_id': ObjectId('5d5c262ca989f250c582fbea'),
 'diesel': '$2.747',
 'mid_grade': '$2.820',
 'premium': '$3.099',
 'regular': '$2.550',
 'state': 'Colorado'}
{'_id': ObjectId('5d5c262ca989f250c582fbeb'),
 'diesel': '$3

In [38]:
# This is how you can print or return all of the entries that match a certain criteria in a MongoDB collection 
# using Pymongo.
for post in db.fuel_prices.find({"regular": "$3.039"}):
...   pprint.pprint(post)

{'_id': ObjectId('5d5c262ba989f250c582fbe5'),
 'diesel': '$2.986',
 'mid_grade': '$3.127',
 'premium': '$3.270',
 'regular': '$3.039',
 'state': 'Alaska'}


In [39]:
# This is how you can count the number of documents in a MongoDB collection using PyMongo.
db.fuel_prices.count_documents({})

51

In [40]:
# This is how you can count the number of documents that match a certain criteria in a MongoDB collection 
# using PyMongo.
db.fuel_prices.count_documents({"regular": "$3.039"})

1

In [56]:
# This is a way that aggregations functions can be performed using PyMongo and bson.son
from bson.son import SON
pipeline = [
    {"$unwind": "$mid_grade"},
    {"$group": {"_id": "$mid_grade", "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1), ("_id", -1)])}
]

pprint.pprint(list(db.fuel_prices.aggregate(pipeline)))

[{'_id': '$2.770', 'count': 2},
 {'_id': '$2.752', 'count': 2},
 {'_id': '$3.820', 'count': 1},
 {'_id': '$3.755', 'count': 1},
 {'_id': '$3.437', 'count': 1},
 {'_id': '$3.358', 'count': 1},
 {'_id': '$3.284', 'count': 1},
 {'_id': '$3.270', 'count': 1},
 {'_id': '$3.162', 'count': 1},
 {'_id': '$3.127', 'count': 1},
 {'_id': '$3.114', 'count': 1},
 {'_id': '$3.104', 'count': 1},
 {'_id': '$3.091', 'count': 1},
 {'_id': '$3.041', 'count': 1},
 {'_id': '$3.030', 'count': 1},
 {'_id': '$3.010', 'count': 1},
 {'_id': '$3.006', 'count': 1},
 {'_id': '$2.991', 'count': 1},
 {'_id': '$2.977', 'count': 1},
 {'_id': '$2.970', 'count': 1},
 {'_id': '$2.952', 'count': 1},
 {'_id': '$2.951', 'count': 1},
 {'_id': '$2.929', 'count': 1},
 {'_id': '$2.926', 'count': 1},
 {'_id': '$2.912', 'count': 1},
 {'_id': '$2.900', 'count': 1},
 {'_id': '$2.898', 'count': 1},
 {'_id': '$2.886', 'count': 1},
 {'_id': '$2.878', 'count': 1},
 {'_id': '$2.842', 'count': 1},
 {'_id': '$2.820', 'count': 1},
 {'_id':

In [57]:
# This is how you can run an explain plan for this aggregation
db.command('aggregate', 'fuel_prices', pipeline=pipeline, explain=True)

{'stages': [{'$cursor': {'query': {},
    'fields': {'mid_grade': 1, '_id': 0},
    'queryPlanner': {'plannerVersion': 1,
     'namespace': 'etl_db.fuel_prices',
     'indexFilterSet': False,
     'parsedQuery': {},
     'winningPlan': {'stage': 'COLLSCAN', 'direction': 'forward'},
     'rejectedPlans': []}}},
  {'$unwind': {'path': '$mid_grade'}},
  {'$group': {'_id': '$mid_grade', 'count': {'$sum': {'$const': 1}}}},
  {'$sort': {'sortKey': {'count': -1, '_id': -1}}}],
 'ok': 1.0}

In [ ]:
db.fuel_prices.aggregate(
   [
     {
       $group:
         {
           _id: "$item",
           avgAmount: { $avg: { $multiply: [ "$price", "$quantity" ] } },
           avgQuantity: { $avg: "$quantity" }
         }
     }
   ]
)